- Pilar Navarro Ramírez
- Antonio Gámiz Delgado

## Tarea 2)

In [9]:
from math import log, floor
from itertools import chain, groupby

alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
n = len(alphabet)
m = int(log(len(alphabet),10))+1
f = '0{0}d'.format(str(m))
chNum = {v:format(i,f) for i, v in enumerate(alphabet)}
numCh = {v:i for i, v in chNum.items()}

alphSpecials = {
            'Á' : 'A',
            'É' : 'E',
            'Í' : 'I',
            'Ó' : 'O',
            'Ú' : 'U',
            'Ä' : 'A',
            'Ë' : 'E',
            'Ï' : 'I',
            'Ö' : 'O',
            'Ü' : 'U',
            'Ñ' : 'GN'
        }

In [10]:
class Message:

    def _flatten(self,listOfLists:list) -> list:
        """Flatten one level of nesting"""
        return chain.from_iterable(listOfLists)
    
    def _rBlanks(self,strng:str) -> str:
        """Removes blanks of a string strng and converts to uppercase"""
        return ''.join(strng.split()).upper()
    
    def _normalize(self,strng:str) -> str:
        """
        Removes blanks spaces of the string 'strng'; then removes accents
        according to 'alphSpecials'. If character 'ñ' occurs in 'strng' then
        'GN' appears in 'accum' as an entry, therefore '_flatten' is needed.
        """
        s = self._rBlanks(strng)
        accum = []
        for ch in s:
            if ch in alphSpecials:
                accum.append(alphSpecials[ch])
            else:
                accum.append(ch)
        return filter(lambda x: x in alphabet,self._flatten(accum))

    def __init__(self,strng):
        x = self._normalize(strng)
        self.content = ''.join(x)
        self.length = len(self.content)
        
    def __str__(self):
        return self.content

In [11]:
from math import ceil
class Vigenere(Message):
    """ Implementation of the Vigenere cryptosystem """
    
    def __init__(self,strng,k):
        Message.__init__(self,strng)
        self.clave=Message(k)

    def _translation(self, c: str, k: str, mode=True) -> str:
        """ Translate c as many letters as k indicates
        
        Args:
            c (str): letter to be translated
            k (str): letter to translate
            mode (bool): indicate if the translation is forward or backward.
        
        Return:
            Letter c translated using k, backward if 'mode' is True, forward
            otherwise.
        """
        if mode:
            return numCh[format((int(chNum[c])-int(chNum[k]))%len(alphabet),f)]
        return numCh[format((int(chNum[c])+int(chNum[k]))%len(alphabet),f)]
    
        
    def encipher(self,mode=True):
        """ Performs enciphering of a string. 
        Use mode 'True' in order to decipher and 'False' for 
        enciphering """
        
        k=self.clave.content
        m=self.content
        while len(k)<len(m):
            k=k+k
        if len(k)>len(m):
            truncar=len(k)-len(m)
            k=k[:-truncar]

        cipher = [self._translation(m[i],k[i],mode) for i in range(len(k))]
        return ''.join(cipher)

In [12]:
k='hola'

In [13]:
M = Vigenere('the secret protects itself',k)

In [14]:
M.content

'THESECRETPROTECTSITSELF'

In [15]:
M.encipher(False)

'AVPSLQCEADCOASNTZWESLZQ'

In [16]:
C=Vigenere(M.encipher(False),k)

In [17]:
C.encipher()

'THESECRETPROTECTSITSELF'

# Tarea 3)


In [18]:
from collections import Counter
from functools import reduce
from math import floor

In [19]:
LETTER_FREQUENCIES = {'A': 0.08167, 'B': 0.01492, 'C': 0.02782, 'D': 0.04253, 'E': 0.12702, 'F': 0.02228, 'G': 0.02015,
               'H': 0.06094, 'I': 0.06966, 'J': 0.00153, 'K': 0.00772, 'L': 0.04025, 'M': 0.02406, 'N': 0.06749,
               'O': 0.07507, 'P': 0.01929, 'Q': 0.00095, 'R': 0.05987, 'S': 0.06327, 'T': 0.09056, 'U': 0.02758,
               'V': 0.00978, 'W': 0.02360, 'X': 0.00150, 'Y': 0.01974, 'Z': 0.00074}

EN_IC = 0.0685

In [20]:
class VigenereAttack():
    def __init__(self, ciphered_text):
        self.M = ciphered_text
        
    def IC(self, message):
        """ Compute the IC of message """
        
        N = len(message)
        frequencies = Counter(message)
        K0 = 0
        for x in alphabet:
            K0 += frequencies[x]*(frequencies[x]-1)
        K0 /= N*(N-1)
        return K0

    def split(self, n):
        """ Split self.M in n part and group the resulting
        parts by columns """
        
        parts = []
        for x in range(n):
            part = ""
            i = x
            while i < len(self.M):
                part += self.M[i]
                i+=n
            parts.append(part)
        return parts

    def factors(self, x):
        """ Compute the factors of x, excluding 1 """
        
        return [i for i in range(2,x+1) if x%i==0]

    def find_all(self, K):
        """ Find the distance between all repeated substrings
        with length K """
        
        found = []
        for index in range(len(self.M)-K):
            substring = self.M[index:index+K]
            tmp_string = self.M
            while substring in tmp_string:
                begin = tmp_string.index(substring)
                tmp_string = self.M[begin+K::]
                if substring in tmp_string:
                    end = tmp_string.index(substring)
                    found.append(end+K-begin)
                break
        return found

    def most_common_factors(self, start, end):
        """ Compute the first six most common factors of the
        distance between all repeated substrings with length 
        greater or equal than start and lower  or equal than 
        end"""
        
        ocurrences = Counter()
        for K in range(start, end):
            for distance in self.find_all(K):
                ocurrences.update(self.factors(distance))
        return ocurrences.most_common(6)

    def compute_ICs(self, start=3, end=11, n=6):
        """ Compute the IC of the parts of self.M splited
        using self.split """
        factorICs = {}
        for K in self.most_common_factors(start, end):
            parts = self.split(K[0])
            ICs = []
            for part in parts:
                ICs.append(self.IC(part))
            factorICs[K[0]] = ICs
        return factorICs
    
    def find_key_length(self):
        """ Compares the IC of each possible key value and return
        the most similar to the IC of the language """
        
        factorICs = self.compute_ICs()
        for key in factorICs.keys():
            averageIC = reduce(lambda x,y: x+y, factorICs[key], 0) / len(factorICs[key])
            if EN_IC - 0.01 < averageIC and averageIC < EN_IC + 0.01:
                return key
    
    def mic(self, text):
        """ Compute the MIC of text """
        freqX = Counter(text)
        result = 0
        for char in alphabet:
            result += freqX[char]*LETTER_FREQUENCIES[char]
        return result / (len(text)*len(alphabet))

    def find_key_letter(self, text):
        """ Find the letter (probably) used to cipher text """
        
        key_letter = ''
        max_corr = 0
        for letter in alphabet:
            shifted = Vigenere(text, letter).encipher()
            corr = self.mic(shifted)
            if corr > max_corr:
                max_corr = corr
                key_letter = letter

        return key_letter

    def find_key(self, n):
        """ Find the key (probably) used to cipher self.M """
        
        letterFreq = Counter(self.M)
        columns = self.split(n)
        key=''
        for column in columns:
            key += self.find_key_letter(column)
        return key
    
    def friedman(self, k0 = 0):
        """ Compute the value of the Friedman test
            
            Args:
                k0 (float): average IC of the parts of the message
        """
        
        KP = 0.0755
        KR = 1 / 27
        N = len(self.M)
        sumFreq = 0
        freq = Counter(self.M)
        for char in freq:
            sumFreq += freq[char]*(freq[char]-1)
        K0 = sumFreq / (N * (N-1)) if k0==0 else k0
        return (KP-KR)/(K0-KR)
    
    def find_key_length_friedman(self):
        """ Find the key length (probably) using the Friedman approach """
        
        K = floor(self.friedman())
        friedmanValues = []
        for k in range(K-2,K+3):
            parts = self.split(k)
            averageIC = 0
            for part in parts:
                averageIC += self.IC(part)
            averageIC /= k
            friedmanValues.append(self.friedman(averageIC))
        optimoIC = min(friedmanValues)
        possibleKeyLength = friedmanValues.index(optimoIC)+K-2
        return possibleKeyLength

# Tarea 4)

In [21]:
def readTxt(file):
    with open(file,'r') as f:
        lines = f.readlines()
    accum = [k[:-1] for k in lines]
    return ''.join(accum)

Leemos el mensaje, y lo almacenamos en M.

In [22]:
M = readTxt('mensaje.txt'); M

'UECWKDVLOTTVACKTPVGEZQMDAMRNPDDUXLBUICAMRHOECBHSPQLVIWOFFEAILPNTESMLDRUURIFAEQTTPXADWIAWLACCRPBHSRZIVQWOFROGTTNNXEVIVIBPDTTGAHVIACLAYKGJIEQHGECMESNNOCTHSGGNVWTQHKBPRHMVUOYWLIAFIRIGDBOEBQLIGWARQHNLOISQKEPEIDVXXNETPAXNZGDXWWEYQCTIGONNGJVHSQGEATHSYGSDVVOAQCXLHSPQMDMETRTMDUXTEQQJMFAEEAAIMEZREGIMUECICBXRVQRSMENNWTXTNSRNBPZHMRVRDYNECGSPMEAVTENXKEQKCTTHSPCMQQHSQGTXMFPBGLWQZRBOEIZHQHGRTOBSGTATTZRNFOSMLEDWESIWDRNAPBFOFHEGIXLFVOGUZLNUSRCRAZGZRTTAYFEHKHMCQNTZLENPUCKBAYCICUBNRPCXIWEYCSIMFPRUTPLXSYCBGCCUYCQJMWIEKGTUBRHVATTLEKVACBXQHGPDZEANNTJZTDRNSDTFEVPDXKTMVNAIQMUQNOHKKOAQMTBKOFSUTUXPRTMXBXNPCLRCEAEOIAWGGVVUSGIOEWLIQFOZKSPVMEBLOHLXDVCYSMGOPJEFCXMRUIGDXNCCRPMLCEWTPZMOQQSAWLPHPTDAWEYJOGQSOAVERCTNQQEAVTUGKLJAXMRTGTIEAFWPTZYIPKESMEAFCGJILSBPLDABNFVRJUXNGQSWIUIGWAAMLDRNNPDXGNPTTGLUHUOBMXSPQNDKBDBTEECLECGRDPTYBVRDATQHKQJMKEFROCLXNFKNSCWANNAHXTRGKCJTTRRUEMQZEAEIPAWEYPAJBBLHUEHMVUNFRPVMEDWEKMHRREOGZBDBROGCGANIUYIBNZQVXTGORUUCUTNBOEIZHEFWNBIGOZGTGWXNRHERBHPHGSIWXNPQMJVBCNEIDVVOAGLPONAPWYPXKEFKOCMQTRTIDZB

Creamos una instancia de VigenereAttack usando M.

In [23]:
V = VigenereAttack(M); V

Buscamos la longitud de la clave usando el test de Friedman:

In [24]:
K_LENGTH = V.find_key_length_friedman(); K_LENGTH

7

También podemos buscar la longitud de la clave usando el test de Kasiski:

In [25]:
K_LENGTH = V.find_key_length(); K_LENGTH

7

Con la longitud de la clave ya conocida, podemos encontrar la clave usada:

In [26]:
K = V.find_key(K_LENGTH); K

'CAPITAN'

Finalmente, podemos descifrar el texto:

In [27]:
Vigenere(M, K).encipher()

'SENORDIJOELCAPITANNEMOMOSTRANDOMELOSINSTRUMENTOSCOLGADOSDELASPAREDESDESUCAMAROTEHEAQUILOSAPARATOSEXIGIDOSPORLANAVEGACIONDELNAUTILUSALIGUALQUEENELSALONLOSTENGOAQUIBAJOMISOJOSINDICANDOMEMISITUACIONYMIDIRECCIONEXACTASENMEDIODELOCEANOALGUNOSDEELLOSLESONCONOCIDOSCOMOELTERMOMETROQUEMARCALATEMPERATURAINTERIORDELNAUTILUSELBAROMETROQUEPESAELAIREYPREDICELOSCAMBIOSDETIEMPOELHIGROMETROQUEREGISTRAELGRADODESEQUEDADDELAATMOSFERAELSTORMGLASSCUYAMEZCLAALDESCOMPONERSEANUNCIALAINMINENCIADELASTEMPESTADESLABRUJULAQUEDIRIGEMIRUTAELSEXTANTEQUEPORLAALTURADELSOLMEINDICAMILATITUDLOSCRONOMETROSQUEMEPERMITENCALCULARMILONGITUDYPORULTIMOMISANTEOJOSDEDIAYDENOCHEQUEMESIRVENPARAESCRUTARTODOSLOSPUNTOSDELHORIZONTECUANDOELNAUTILUSEMERGEALASUPERFICIEDELASAGUASSONLOSINSTRUMENTOSHABITUALESDELNAVEGANTEYSUUSOMEESCONOCIDOREPUSEPEROHAYOTROSAQUIQUERESPONDENSINDUDAALASPARTICULARESEXIGENCIASDELNAUTILUSESECUADRANTEQUEVEORECORRIDOPORUNAAGUJAINMOVILNOESUNMANOMETROESUNMANOMETROENEFECTOPUESTOENCOMUNICACIONCONELAGUACUYAPRESIONEXTERIORI